I changed the parameters by referring to the following
https://www.kaggle.com/code/paulorzp/median-model-jpx

The target distribution is not close to the normal distribution because the mean is slightly higher than the median. Confirmation is necessary

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:

import numpy as np
import pandas as pd


def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
starts = 20

for item in range(36):
    NDAYS = starts + item
    lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)
    pred = lastdays.groupby("SecuritiesCode")["Target"].mean().rank(ascending=False,method="first").astype(int) -1
    pred = pred.reset_index()
    pred = dict(zip(pred["SecuritiesCode"],pred["Target"]))
    
    # Computing the score for each NDAYS to see the possible highest scores
    stock_prices2 = prices.loc[prices["Date"]>= prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)
    stock_prices2["Rank"] = stock_prices2.groupby("Date")["Target"].rank(ascending=False,method="first") -1 
    stock_prices2["Rank"] = stock_prices2["Rank"].astype("int")
    score = calc_spread_return_sharpe(stock_prices2, portfolio_size= 200, toprank_weight_ratio= 2)
    
    print('NDAYS: ' + str(NDAYS) + ' score: ' + str(score))

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
    env.predict(sample_prediction)